In [12]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math

In [13]:
mr_file = "../data/didi/mr.txt"

In [14]:
df = pd.read_csv(mr_file, sep=";")

In [15]:
def extract_cpath(cpath):
    if (isinstance(cpath, float)):
        return []
    if (cpath ==''):
        return []
    return [int(s) for s in cpath.split(',')]

In [16]:
df["cpath_list"] = df.apply(lambda row: extract_cpath(row.cpath), axis=1)

In [17]:
all_edge_ids = np.unique(np.hstack(df.cpath_list)).tolist()

In [18]:
network_gdf = gpd.read_file("../data/didi/edges.shp")
network_gdf.id = network_gdf.fid.astype(int)
network_gdf.head()

/home/morningstarwang/anaconda3/envs/tf2-cpu/lib/python3.7/site-packages/geopandas/geodataframe.py:182: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


,osmid,ref,highway,oneway,length,bridge,lanes,name,maxspeed,tunnel,junction,width,access,u,v,key,fid,geometry
0,258246480,G108;G318,trunk,False,71.791,None,None,None,None,None,None,None,None,6432489476,5742830014,0,0,"LINESTRING (103.42044 30.33845, 103.42074 30.3..."
1,258246480,G108;G318,trunk,False,353.395,None,None,None,None,None,None,None,None,6432489476,6432489512,0,1,"LINESTRING (103.42044 30.33845, 103.41932 30.3..."
2,686279042,None,residential,False,170.951,None,None,None,None,None,None,None,None,6432489476,6432489503,0,2,"LINESTRING (103.42044 30.33845, 103.41975 30.3..."
3,"[738608224, 738608222, 738608223]",None,residential,False,66.735,yes,None,None,None,None,None,None,None,6916407301,6916407304,0,3,"LINESTRING (104.06211 30.47650, 104.06233 30.4..."
4,686279043,None,residential,False,177.733,None,None,None,None,None,None,None,None,6432489478,6432489480,0,4,"LINESTRING (103.41484 30.33463, 103.41377 30.3..."


In [19]:
edges_df = network_gdf[network_gdf.id.isin(all_edge_ids)].reset_index()

In [20]:
edges_df["points"] = edges_df.apply(lambda row: len(row.geometry.coords), axis=1)

In [24]:
edges_df.head()


,index,osmid,ref,highway,oneway,length,bridge,lanes,name,maxspeed,tunnel,junction,width,access,u,v,key,fid,geometry,points
0,178,740880879,None,trunk_link,True,310.176,yes,None,None,None,None,None,None,None,6936330326,6936330330,0,178,"LINESTRING (104.07742 30.72484, 104.07625 30.7...",5
1,182,611517553,中环路,primary,True,77.244,None,None,中环路昭觉寺横路段,None,None,None,None,None,6011355225,1504213677,0,182,"LINESTRING (104.10162 30.70328, 104.10159 30.7...",3
2,183,637666355,None,tertiary_link,True,66.179,None,None,None,None,None,None,None,None,6011355225,4534007689,0,183,"LINESTRING (104.10162 30.70328, 104.10162 30.7...",12
3,188,740880884,None,trunk_link,True,293.505,yes,None,None,None,None,None,None,None,6936330330,6936330360,0,188,"LINESTRING (104.07436 30.72577, 104.07396 30.7...",9
4,193,723512138,None,secondary,True,11.818,None,None,建设北路一段,None,None,None,None,None,2712272990,5459472400,0,193,"LINESTRING (104.09578 30.67463, 104.09590 30.6...",2


In [22]:
edges_df[["fid","u","v","geometry","points"]].to_csv("../data/didi/matched_edges.csv",sep=";",index=False)